<a href="https://colab.research.google.com/github/avlntav/projects/blob/main/Python_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Module 1 Python**



The below code and logic solves all the questions asked in Module 1

In [ ]:
##Module 1
import os
import shutil

def move_files(source_folder, destination_folder, bad_folder):
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        bad_path = os.path.join(bad_folder, filename)

        # Check if file already exists in destination folder
        if os.path.isfile(destination_path):
            print(f"File '{filename}' already exists in destination folder.")
        else:
            # Check if file is a CSV and not empty
            if filename.lower().endswith('.csv') and os.path.getsize(source_path) > 0:
                shutil.move(source_path, destination_path)
                print(f"Moved '{filename}' to destination folder.")
            else:
                shutil.move(source_path, bad_path)
                print(f"Moved '{filename}' to bad folder.")

# Example usage:
source_folder = '/content/source'
destination_folder = '/content/destination'
bad_folder = '/content/Bad_files'

move_files(source_folder, destination_folder, bad_folder)




Moved 'Areas_in_blore.xlsx' to bad folder.
Moved 'data_file_20210527182730.csv' to destination folder.
Moved 'data_file_20210528182554.csv' to destination folder.
Moved 'data_file_20210528182844.csv' to destination folder.


**Module 2**

The below code converts multiple mobile numbers in Phone colume to separate columns "Contact1" and "Contact2". Any null records that do not have a primary mobile number, or having null values in primary fields are considered as bad records. all the bad records are pushed to another Pipe separated file

**NOTE:** The files are saved in Tab delimiter because excel is parsing csv format in a bad way, so for better visualization of the data in excel file Pipe separator is used.


In [123]:
import os
import pandas as pd
import re

# Function to clean phone numbers
def clean_phone(phone):
    phone = str(phone).replace('+', '').strip()
    phone = str(phone).replace(' ', '').strip()
    if 10 <= len(phone) < 14:
        return phone
    else:
        return None  # Return None for bad records

# Function to check if the value is valid for online_order column
def is_valid_online_order(value):
    return value.lower() in ['yes', 'no']

# Function to check if the value is valid for book_table column
def is_valid_book_table(value):
    return value.lower() in ['yes', 'no']

# Function to remove special characters from a string
def remove_special_chars(text):
    return re.sub(r'[^\w\s]', '', text)

# Folder containing Excel files
folder_path = '/content/destination'
dest_path = '/content/source'

# List to store bad records
bad_records = []

# Iterate through each Excel file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # Assuming Excel files have .csv extension
        file_path = os.path.join(folder_path, file_name)
        # Read Excel file into a DataFrame
        df = pd.read_csv(file_path)

        # Remove special characters from address and reviews_list columns
        df['address'] = df['address'].apply(remove_special_chars)
        df['reviews_list'] = df['reviews_list'].apply(remove_special_chars)

        df['phone'] = df['phone'].str.replace( '\r\n', '\n')


        split_phones = df['phone'].str.split(r'\n', expand=True)

        # Assign the first two columns to 'contact1' and 'contact2'
        df['contact1'] = split_phones[0]
        df['contact2'] = split_phones[1]

        # Clean phone numbers
        df['contact1'] = df['contact1'].apply(clean_phone)
        df['contact2'] = df['contact2'].apply(clean_phone)

        # Convert contact1 and contact2 columns to object/string type
        df['contact1'] = df['contact1'].fillna(' ').astype(str)
        df['contact2'] = df['contact2'].fillna(' ').astype(str)

        # Separate bad records for phone numbers
        bad_records.extend(df[df['contact1'].isnull()].to_dict('records'))

        # Check for bad records in online_order column
        bad_records.extend(df[~df['online_order'].apply(is_valid_online_order)].to_dict('records'))

        # Check for bad records in book_table column
        bad_records.extend(df[~df['book_table'].apply(is_valid_book_table)].to_dict('records'))

        # Check for null values in rate column
        bad_records.extend(df[df['rate'].isnull()].to_dict('records'))

        # Check for null values in location column
        bad_records.extend(df[df['location'].isnull()].to_dict('records'))

        # Remove bad records from DataFrame
        df = df.dropna(subset=['contact1', 'rate', 'location'])
        df = df[df['online_order'].apply(is_valid_online_order)]
        df = df[df['book_table'].apply(is_valid_book_table)]

        # Save cleaned records to a new csv file with pipe delimiter
        cleaned_file_name = file_name.replace('.csv', '_cleaned.csv')
        cleaned_file_path = os.path.join(dest_path, cleaned_file_name)

        # Converting the Contact information into String type


        df.to_csv(cleaned_file_path, sep='|', index=False)

# Create DataFrame for bad records
bad_records_df = pd.DataFrame(bad_records)

# Convert contact1 and contact2 columns to object/string type in bad records DataFrame
bad_records_df['contact1'] = bad_records_df['contact1'].fillna(' ').astype('str')
bad_records_df['contact2'] = bad_records_df['contact2'].fillna(' ').astype('str')

# Save bad records to CSV file with pipe delimiter
bad_records_csv = os.path.join(dest_path, 'bad_records.csv')
bad_records_df.to_csv(bad_records_csv, sep='|', index=False)

print("Cleaning complete!")


Cleaning complete!


In [124]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 50)

In [129]:
processed_file_example = pd.read_csv("/content/source/data_file_20210527182730_cleaned.csv", sep = "|")


In [130]:
#Displaying sample data (50 records) from the cleaned data file
Bad_file = pd.read_csv("/content/source/bad_records.csv", sep = "|")
